In [0]:
import numpy as np

# <pad>  - 0
# <unk>  - 1 
# <sos>  - 2 
# <eos>  - 3
# word A - 4
# word B - 5

# batch targets (batch size, batch length)
bts = np.array([
[2, 1, 4, 3, 0],  # target 1
[2, 1, 4, 5, 3],  # target 2
[2, 4, 3, 0, 0]   # target 3
])
# batch predictions (batch size, batch length)
bps = np.array([
[2, 1, 5, 1, 0],  # prediction 1
[2, 1, 5, 4, 3],  # prediction 2
[2, 4, 3, 0, 0]   # prediction 3
])

In [13]:
# set (unique) metrics (without order)
for p, t in zip(bts, bps):
  t_set = set(t)  # unique labels
  t_set -= {0, 1, 2, 3}  # remove special labels
  eos = p[p == 3][0] if 3 in p else len(p)  # find first <eos> in prediction
  p_eos = p[:eos]  # cut prediction to first <eos>
  p_set = set(p_eos)  # unique labels
  p_set -= {0, 1, 2, 3}  # remove special labels
  i_set = t_set.intersection(p_set)  # shared labels
  pre = len(i_set)/len(p_set) if len(p_set) != 0 else 0.  # precision
  rec = len(i_set)/len(t_set) if len(t_set) != 0 else 0.  # recalll
  f1 = 2*pre*rec/(pre + rec) if pre + rec != 0 else 0.  # f1
  print(f'target: {t}, prediction: {p}, cut prediction: {p_eos}')
  print(f'target set: {t_set}, prediction set: {p_set}')
  print(f'precision: {pre}, recall: {rec}, f1: {f1}\n')


target: [2 1 5 1 0], prediction: [2 1 4 3 0], cut prediction: [2 1 4]
target set: {5}, prediction set: {4}
precision: 0.0, recall: 0.0, f1: 0.0

target: [2 1 5 4 3], prediction: [2 1 4 5 3], cut prediction: [2 1 4]
target set: {4, 5}, prediction set: {4}
precision: 1.0, recall: 0.5, f1: 0.6666666666666666

target: [2 4 3 0 0], prediction: [2 4 3 0 0], cut prediction: [2 4 3]
target set: {4}, prediction set: {4}
precision: 1.0, recall: 1.0, f1: 1.0

